# 1. Install and Import Dependencies

In [1]:
## For ARM Mac ##
# !pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
# For Linux
# !pip3 install torch torchvision torchaudio

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu


In [ ]:
import wandb
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import uuid   # Unique identifier
import os
import time
import sys
from zipfile import ZipFile

## Data set and modified yolo repo downloaded from kaggle

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"","key":""} #Place your kaggle credentials here
import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mak1999/yolov5-drowsy

In [ ]:
filename = 'yolov5-drowsy.zip'
with ZipFile(filename, 'r') as z:
    print('Extracting all the files now...')
    z.extractall()
    print('Done!')
os.remove(filename)

In [12]:
!pip3 install -r yolov5/requirements.txt

# 2. Train from scratch

# Image Labelling

In [35]:
!git clone https://github.com/tzutalin/labelImg

Cloning into 'labelImg'...
remote: Enumerating objects: 2097, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 2097 (delta 0), reused 4 (delta 0), pack-reused 2090
Receiving objects: 100% (2097/2097), 237.14 MiB | 802.00 KiB/s, done.
Resolving deltas: 100% (1242/1242), done.


In [ ]:
#Installing some dependencies
# brew install qt  # Install qt-5.x.x by Homebrew
# brew install libxml2

# or using pip

# pip3 install pyqt5 lxml # Install qt and lxml by pip

#Inside LabelImg dir
# make qt5py3
# python3 labelImg.py
# python3 labelImg.py [IMAGE_PATH] [PRE-DEFINED CLASS FILE]

In [4]:
!pip3 install pyqt5 lxml

In [40]:
%cd labelImg
!pyrcc5 -o libs/resources.py resources.qrc
!make qt5py3
!python3 labelImg.py
%cd ..

/Users/bitanguharoy/Downloads/yolo-drowsiness-detection/labelImg
zsh:1: command not found: pyrcc5
pyrcc5 -o libs/resources.py resources.qrc
make: pyrcc5: No such file or directory
make: *** [qt5py3] Error 1
Traceback (most recent call last):
  File "/Users/bitanguharoy/Downloads/yolo-drowsiness-detection/labelImg/labelImg.py", line 13, in <module>
    from PyQt5.QtGui import *
ModuleNotFoundError: No module named 'PyQt5'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bitanguharoy/Downloads/yolo-drowsiness-detection/labelImg/labelImg.py", line 22, in <module>
    import sip
ModuleNotFoundError: No module named 'sip'
/Users/bitanguharoy/Downloads/yolo-drowsiness-detection


# Training with different models

### Installing wandb for tracking

In [3]:
# !pip3 install wandb
# !wandb login --relogin
os.environ['WANDB_ENTITY'] = 'ipda526'
os.environ['WANDB_PROJECT'] = 'yolov5-drowsiness-detection'
wandb.login(key='') #Place your wandb api key here

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/bitanguharoy/.netrc


True

## Training on config which gave the best results on our UTA-RLDD dataset

In [3]:
%cd yolov5
!python3 train.py \
--img 640 \
--batch 16 \
--epochs 400 \
--data "drowsy_dataset.yaml" \
--hyp "data/hyps/hyp.no-augmentation.yaml" \
--optimizer 'SGD' \
--weights "yolov5s.pt" \
--patience 75 \
--workers 2 \
--save-period 25 
%cd ..

/Users/bitanguharoy/Downloads/yolo-drowsiness-detection/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Currently logged in as: bitanguharoy (ipda526). Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=drowsy_dataset.yaml, hyp=data/hyps/hyp.no-augmentation.yaml, epochs=400, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=75, freeze=[0], save_period=25, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github

In [4]:
!wandb sweep --help

Usage: wandb sweep [OPTIONS] CONFIG_YAML_OR_SWEEP_ID

  Create a sweep

Options:
  -p, --project TEXT        The project of the sweep.
  -e, --entity TEXT         The entity scope for the project.
  --controller              Run local controller
  --verbose                 Display verbose output
  --name TEXT               Set sweep name
  --program TEXT            Set sweep program
  --update TEXT             Update pending sweep
  -c, --launch_config FILE  Path to JSON or YAML file which defines how to
                            launch the sweep.
  --stop                    Finish a sweep to stop running new runs and let
                            currently running runs finish.
  --cancel                  Cancel a sweep to kill all running runs and stop
                            running new runs.
  --pause                   Pause a sweep to temporarily stop running new
                            runs.
  --resume                  Resume a sweep to continue running new runs.
  -q,

In [7]:
%cd yolov5
!wandb sweep wandb_config.yaml
%cd ..

/Users/bitanguharoy/Downloads/yolo-drowsiness-detection/yolov5
wandb: Creating sweep from: wandb_config.yaml
wandb: Created sweep with ID: okwyr5bx
wandb: View sweep at: https://wandb.ai/ipda526/yolov5-drowsiness-detection/sweeps/okwyr5bx
wandb: Run sweep agent with: wandb agent ipda526/yolov5-drowsiness-detection/okwyr5bx
/Users/bitanguharoy/Downloads/yolo-drowsiness-detection


In [11]:
%cd yolov5
!wandb agent --count 5 ipda526/yolov5-drowsiness-detection/okwyr5bx #This is the sweep id got from previous step
%cd ..

# 3. Model Testing

In [5]:
model_url = 'ipda526/yolov5-drowsiness-detection/run_ol2p2qop_model:v15' #from wandb artifacts
run = wandb.init()
artifact = run.use_artifact(model_url, type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=f'{artifact_dir}/best.pt', force_reload=True)

## Realtime testing in video-cam

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()